In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn

In [0]:

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 10
batch_size = 100
learning_rate = 0.001

# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [0]:
# Fully Connected Neural Network
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, num_classes)
 
    
    def forward(self, x):
      out = self.fc1(x)
      out = self.relu1(out)
      out = self.fc2(out)
      out = self.relu2(out)
      out = self.fc3(out)

      return out


In [0]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)


In [0]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [0]:


for epoch in range(num_epochs):
  for images, labels in train_loader:
    images=images.reshape(-1, 28*28)
    outputs=model(images)
    
    loss=criterion(outputs, labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
  print('epoch: [{}/{}], loss:{}'.format(epoch, num_epochs, loss.item()))

epoch: [0/10], loss:0.009656090289354324
epoch: [1/10], loss:0.007865109480917454
epoch: [2/10], loss:0.012163043022155762
epoch: [3/10], loss:0.004481334704905748
epoch: [4/10], loss:0.06491880863904953
epoch: [5/10], loss:0.0023711109533905983
epoch: [6/10], loss:0.004776086658239365
epoch: [7/10], loss:0.0005008220905438066
epoch: [8/10], loss:0.021013446152210236
epoch: [9/10], loss:0.00035291671520099044


In [0]:
with torch.no_grad():
  correct=0
  total=0
  for images, labels in test_loader:
    images=images.reshape(-1, 28*28)
    
    outputs=model(images)
    prediction=torch.max(outputs,1)[1]
    correct+=(prediction==labels).sum().item()
    total+=batch_size
  print('Accuracy:{}%'.format(correct/total))

Accuracy:0.9822%


In [0]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')